In [6]:
import requests, re, pprint, os

from selenium import webdriver

from selenium.webdriver.common.by import By

from time import sleep

import json, os, random


listBooks = [] # 書籍的中文名、網址的dict

dr = webdriver.Chrome('./chromedriver')

pat1 = r'(/ebooks/(\d{5}))\"\>([\u4E00-\u9FFF]+)' # 書籍網址
pat2 = r'/files/[0-9]+/[0-9]+(.{2})?.txt' # 書籍文字檔下載網址
pat3 = r'[\u4E00-\u9FFF，。：「」；、？！『』]+' # 中文字

# 自訂標頭
my_hs = { 
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'
}

# 連上古騰堡中文書清單
response = requests.get('https://www.gutenberg.org/browse/languages/zh')

# 建立文放書籍的資料夾
folderPath = 'gutenberg_zh'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

# 蒐集書籍的中文名、網址
mI = re.finditer(pat1, response.text)
for index, m in enumerate(mI):
    if index <= 9: # 可以自訂要找幾本
        listBooks.append({
            "link": m.group(1),
            "title": m.group(3)
        })
# pprint.pprint(f'listBooks: {listBooks}')


for index, link in enumerate(listBooks):
    ti = random.random() * 2
    
    dr.get('http://www.google.com/') 
    docLink = f'https://www.gutenberg.org{link["link"]})'# 將片段的書籍網址弄完整
#     print(f'docLink: {docLink}')
    docLink = docLink.replace(')','') # 去除古騰堡阻擋爬蟲加上的括號
#     print(f'docLink: {docLink}')
    dr.get(docLink) # 用selenium進入書籍網址
    sleep(ti)
    
    urls = dr.find_elements(By.CSS_SELECTOR, "#download > div > table > tbody a") # 找書籍下載網址所在標籤
    for aHref in urls:
        aLink = aHref.get_attribute('href') # 找書籍下載網址
#         print(f"aLink: {aLink}")
        bLinks = re.finditer(pat2, aLink) # 找到書籍文字檔下載網址
        for bLink in bLinks:
            cLink = bLink.group() # re找到的東西是集合，需要一個變數接資料
#     print(f'cLink: {cLink}')

    textLink = f'https://www.gutenberg.org{cLink}' # 將片段的書籍文字檔下載網址弄完整
    print(f'textLink: {textLink}')
    textResponse = requests.get(textLink) # 用requests進入書籍指定類型下載網址
    textResponse.encoding = 'utf-8' # 用utf-8讀取該網頁，要不然預設是gbk，無法呈現中文

    texts = re.finditer(pat3, textResponse.text) # 過濾掉非中文
#     print(textResponse.text)
    
#     zhs = '' # 中文片段
    content = [] # 中文片段的list，在迴圈外歸零，壁面囤積上一本書的內容
#     contents = '' # 文章內文
    for text in texts:
        zhs = text.group() # 取得中文片段
        content.append(zhs) # 將中文片段存到list
    contents = '\n'.join(content) # 將list組成字串，每個片段以換行隔開
#     print(f'text: {text}')
#     print(f'zhs: {zhs}')
#     print(contents)
    
    titleName = link["title"] #取得書名
    print(f'titleName: {titleName}')
    fp = open(f'./gutenberg_zh/{titleName}.txt', 'w', encoding = 'utf-8')
    fp.write(contents)
    fp.close

# Fifth

textLink: https://www.gutenberg.org/files/25328/25328-0.txt
titleName: 豆棚閒話
textLink: https://www.gutenberg.org/files/24225/24225-0.txt
titleName: 戲中戲
textLink: https://www.gutenberg.org/files/27119/27119-0.txt
titleName: 比目魚
textLink: https://www.gutenberg.org/files/24185/24185-0.txt
titleName: 比目魚
textLink: https://www.gutenberg.org/files/12479/12479-0.txt
titleName: 三字經
textLink: https://www.gutenberg.org/files/25146/25146-0.txt
titleName: 山水情
textLink: https://www.gutenberg.org/files/25288/25288-0.txt
titleName: 山海經
textLink: https://www.gutenberg.org/files/23825/23825-0.txt
titleName: 施公案
textLink: https://www.gutenberg.org/files/25393/25393-0.txt
titleName: 施公案
textLink: https://www.gutenberg.org/files/25501/25501-0.txt
titleName: 易經
